In [1]:
import praw

In [2]:
reddit = praw.Reddit(client_id='wDYqaMq-X2YCmUJTqPCaqw',
                     client_secret='IqRTJP1nRwAqOpvu8oRhigLC7ZdRgg',
                     user_agent='Islam_final_project')

In [ ]:

related_subreddits = '+'.join(['economy', 'finance', 'business', 'investing'])

def stream_headlines_to_file(subreddits, max_headlines, output_file):
    subreddit = reddit.subreddit(subreddits)
    collected_headlines = set()

    with open(output_file, 'w', encoding='utf-8') as file:
        for submission in subreddit.stream.submissions():
            collected_headlines.add(submission.title)
            file.write(submission.title + '\n')

            if len(collected_headlines) >= max_headlines:
                break

    print(f"Total headlines collected: {len(collected_headlines)}")
    print(f"Headlines written to: {output_file}")

output_file_path = 'headlines.txt'
max_headlines = 10000

stream_headlines_to_file(related_subreddits, max_headlines, output_file_path)

In [6]:
def collect_comments(subreddits, min_comment_words, max_comments):
    subreddit = reddit.subreddit(subreddits)
    collected_comments = []

    for submission in subreddit.stream.submissions():
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            if len(comment.body.split()) > min_comment_words:
                collected_comments.append(comment.body)

                if len(collected_comments) >= max_comments:
                    return collected_comments

    return collected_comments

def export_comments_to_file(comments, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        for comment in comments:
            file.write(comment + '\n')

    print(f"Total comments collected: {len(comments)}")
    print(f"Comments written to: {output_file}")

output_comments_file_path = 'comments.txt'
min_comment_words = 15
max_comments = 10000


In [7]:
collected_comments = collect_comments(related_subreddits, min_comment_words, max_comments)

KeyboardInterrupt: 

In [8]:
export_comments_to_file(collected_comments, output_comments_file_path)

NameError: name 'collected_comments' is not defined

----
----

In [ ]:
!pip install asyncpraw
!pip install nest_asyncio

import asyncio
import asyncpraw
import pandas as pd

async def fetch_comments(subreddit_names, num_comments_per_topic, total_comments, batch_size, output_file, delay_threshold=100, delay_seconds=5):
    
    reddit = asyncpraw.Reddit(
        client_id='wDYqaMq-X2YCmUJTqPCaqw',
        client_secret='IqRTJP1nRwAqOpvu8oRhigLC7ZdRgg',
        user_agent='Islam_final_project',
    )

    data = {'Title': [], 'Comment': [], 'Upvote Ratio': []}
    comments_count = 0

    for subreddit_name in subreddit_names:
        subreddit = await reddit.subreddit(subreddit_name)

        async for submission in subreddit.hot(limit=None):  # Fetch all hot submissions
            print(f"Fetching comments from {subreddit_name}: {submission.title}")

            await submission.load()

            submission.comment_sort = 'top'
            async for comment in submission.comments:
                if isinstance(comment, asyncpraw.models.reddit.more.MoreComments):
                    continue  # Skip MoreComments objects

                data['Title'].append(submission.title)
                data['Comment'].append(comment.body)
                data['Upvote Ratio'].append(submission.upvote_ratio)
                comments_count += 1

                if comments_count >= total_comments:
                    break  # Exit if the desired number of comments is reached

                # Write to the file in batches
                if comments_count % batch_size == 0:
                    write_to_csv(data, output_file)
                    data = {'Title': [], 'Comment': [], 'Upvote Ratio': []}

                # Introduce a delay after every 100 comments
                if comments_count % delay_threshold == 0:
                    print(f"Sleeping for {delay_seconds} seconds...")
                    await asyncio.sleep(delay_seconds)

    # Write any remaining data to the file
    if data['Title']:
        write_to_csv(data, output_file)

def write_to_csv(data, output_file):
    df = pd.DataFrame(data)
    df.to_csv(output_file, mode='a', index=False, header=not pd.DataFrame().empty)

async def main():
    related_subreddits = ['politics']
    num_comments_per_topic = 10  # Change this to the number of comments per topic
    total_comments = 10000  # Change this to the desired total number of comments
    batch_size = 1000  # Change this to the size of each batch
    output_file = '/content/reddit_comments.csv'  # Change this to the desired output file path
    delay_threshold = 100  # Change this to the number of comments before introducing a delay
    delay_seconds = 5  # Change this to the duration of the delay in seconds

    await fetch_comments(related_subreddits, num_comments_per_topic, total_comments, batch_size, output_file, delay_threshold, delay_seconds)

# Run the event loop
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
